In [27]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Test Notebook for Training on the FileTreeSet with the new gaussian readout

In [19]:
import datajoint as dj




import os
import torch
import numpy as np
import pickle 

import nnfabrik
from nnfabrik import main, builder

import nnvision
from numpy import linalg
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import itertools
from neuralpredictors.data.datasets import FileTreeDataset
from neuralpredictors.data.samplers import RepeatsBatchSampler
from torch.utils.data import DataLoader

# Get Dataloader

### For a single file

In [20]:
paths = [str(p) for p in Path('/data/mouse/toliaslab/static/').glob('*22564*/') if p.is_dir()]
paths

['/data/mouse/toliaslab/static/static22564-3-12-preproc0']

In [21]:
paths = [str(p) for p in Path('/data/mouse/toliaslab/static/').glob('*22564*/') if p.is_dir()]
paths
dataset_fn = 'nnvision.datasets.mouse_static_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    normalize=True,
    file_tree=True,
    areas=('V1','AL', 'LM','LI','PM'), 
    layer=('L4', 'L2/3')
)
dataloaders = builder.get_data(dataset_fn, dataset_config)

### Multi Files:


if multiple files should be used for training with matching IDs, use the `mouse_shared_static_loaders` instead, with the list of paths

# Get NNkonsti Model

In [22]:
model_fn = 'nnvision.models.se_core_full_gauss_readout'
model_config = {
   'pad_input': False,
   'stack': -1,
   'layers':4,
   'input_kern': 9,
   'gamma_input': 6.3831,          
   'gamma_readout': 0.0076,
   'hidden_dilation': 1,
   'hidden_kern': 7,
   'hidden_channels': 64,
   'n_se_blocks': 0,
   'depth_separable': True,

   'grid_mean_predictor':
       {
           'type': 'cortex',
           'input_dimensions': 2,
           'hidden_layers':0,
           'hidden_features':20,
           'final_tanh': False,
       },
   'share_features': False,
   'share_grid': False,
}
model = builder.get_model(model_fn=model_fn, 
                          model_config=model_config, 
                          dataloaders=dataloaders, 
                          seed=1000)
model

Encoder(
  (core): SE2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (spatial_conv): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=64, bias=False)
          (out_depth_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (norm): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): ELU(alpha=1.0, inplace=True)
      )
      (layer2): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (i

# Get Trainer

In [23]:
trainer_fn = 'nnvision.training.nnvision_trainer'
trainer_config = dict(max_iter=100,
                      verbose=False, 
                      lr_decay_steps=4,
                      avg_loss=False, 
                      patience=5,
                      lr_init=.0045)
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [ ]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1000)